# Descarga de imagenes

In [1]:
import ee
import geemap
from fastkml import kml
from fiona.drvsupport import supported_drivers
import fiona
import folium
import geojson
import geopandas as gpd
import os
import io
#from geetools import batch
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import shape
import requests
import zipfile

## Geometria

In [2]:
#procesamiento de la geometria
supported_drivers['KML'] = 'rw'

geometriajson = gpd.read_file("Aestudio84.kml", driver="KML")["geometry"]    
geometria = [fila[:2] for fila in list(geometriajson.exterior[0].coords)]
geometria

[(-75.0000007098957, -36.00000411199979),
 (-66.99999928900941, -35.9999999178922),
 (-66.9999986557307, -16.99999959548663),
 (-75.00002582351469, -16.99999299012777),
 (-75.0000007098957, -36.00000411199979)]

In [3]:
#geometriajson.exterior[0].xy
type(geometriajson)

geopandas.geoseries.GeoSeries

In [4]:
#codigo para convertir el kmz a kml
#en caso que no este presente
path_kmz = "Aestudio84.kmz"
kml_doc = "doc.kml"

with zipfile.ZipFile(path_kmz, 'r') as kmz:
    if kml_doc in kmz.namelist():
        kml_content = kmz.read(kml_doc).decode('utf-8')

        with open("Aestudio84.kml", 'w', encoding='utf-8') as kml_file:
            kml_file.write(kml_content)

        print("Archivo kml guardado")
    else:
        print("no se encontro el kml")

Archivo kml guardado


In [5]:
#Hardcodeo de datos
#fecha: cada dia del 2022
##observacion: probar con un dia
inicio = "2022-01-01"
fin = '2022-01-02'

## carga

In [7]:
ee.Initialize(project="sergio-ramrez-agua-de-niebla")
preferencias = {"center": [-27.3666, -70.3322], "zoom": 5.4, "basemap": "Esri.WorldImagery",
                #"data_ctrl":False, "toolbar_ctrl":False, "draw_ctrl":False
               }
map = geemap.Map(**preferencias)
goes = ee.ImageCollection("NOAA/GOES/16/MCMIPF").filterDate(inicio, fin)
atacama = goes.filter(ee.Filter.bounds(geometry=ee.Geometry.Polygon(geometria)))
vis = {
    "bands": ["CMI_C04"],
    "min": 0.0,
    "max": 1.3,
    "gamma": 0.1
}

map.add_layer(atacama, vis, "Zona Agua de Niebla")
map

Map(center=[-27.3666, -70.3322], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

In [12]:
loc = ee.Geometry.Point(-99.2222, 46.7816)
type(loc)

ee.geometry.Geometry

In [8]:
from shapely.geometry import Point
hola = gpd.GeoSeries(Point(punto) for punto in geometria).to_json()
hola

'{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {}, "geometry": {"type": "Point", "coordinates": [-75.0000007098957, -36.00000411199979]}, "bbox": [-75.0000007098957, -36.00000411199979, -75.0000007098957, -36.00000411199979]}, {"id": "1", "type": "Feature", "properties": {}, "geometry": {"type": "Point", "coordinates": [-66.99999928900941, -35.9999999178922]}, "bbox": [-66.99999928900941, -35.9999999178922, -66.99999928900941, -35.9999999178922]}, {"id": "2", "type": "Feature", "properties": {}, "geometry": {"type": "Point", "coordinates": [-66.9999986557307, -16.99999959548663]}, "bbox": [-66.9999986557307, -16.99999959548663, -66.9999986557307, -16.99999959548663]}, {"id": "3", "type": "Feature", "properties": {}, "geometry": {"type": "Point", "coordinates": [-75.00002582351469, -16.99999299012777]}, "bbox": [-75.00002582351469, -16.99999299012777, -75.00002582351469, -16.99999299012777]}, {"id": "4", "type": "Feature", "properties": {}, "geo

In [9]:
out_dir = os.path.expanduser("~/Desktop/goes-images/")

geemap.download_ee_image_collection(atacama.filterBounds(hola), out_dir, scale=10)

EEException: Invalid GeoJSON geometry.

In [ ]:
def func_xmg(image):
  NUM_BANDS = 33
  bands = Array(NUM_BANDS)
  for i in range(1, 17, 1):
    bandName = 'CMI_C' + (100 + i + '').slice(-2)
    offset = ee.Number(image.get(bandName + '_offset'))
    scale =  ee.Number(image.get(bandName + '_scale'))
    bands[(i-1) * 2] = image.select(bandName).multiply(scale).add(offset)
    dqfName = 'DQF_C' + (100 + i + '').slice(-2)
    bands[(i-1) * 2 + 1] = image.select(dqfName)

  return ee.Image(ee.Image(bands).copyProperties(image, image.propertyNames())).toFloat()

scaleOffset = unsigned16.map(func_xmg)
